In [1]:
!pip install dask[complete]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 74.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 111.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 80.8 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np

n = 5_000_000

df = pd.DataFrame({
    "id":np.arange(n),
    "value":np.random.normal(100,20,size=n),
    "category":np.random.choice(['A','B','C'],size=n)
})

#zapis do pliku
df.to_csv("big_data.csv",index=False)

In [3]:
#wczytanie danych z użyciem deskim
import dask.dataframe as dd

ddf = dd.read_csv("big_data.csv")

print(ddf.head(10))

   id       value category
0   0  110.519433        B
1   1  115.264530        B
2   2   95.709197        C
3   3  113.470625        C
4   4   63.662117        B
5   5   78.334256        A
6   6   89.510007        A
7   7   78.565246        B
8   8   69.207758        B
9   9   98.573735        B


In [5]:
#proste operacje
mean_val = ddf["value"].mean()
print(f"Średnia (leniwa): {mean_val}")

print(f"Średnia (obliczona): {mean_val.compute()}")

Średnia (leniwa): <dask_expr.expr.Scalar: expr=ArrowStringConversion(frame=FromMapProjectable(1626709))['value'].mean(), dtype=float64>
Średnia (obliczona): 100.00710789591383


In [7]:
grouped = ddf.groupby("category")["value"].mean()
print(f"Średnia per kategoria(leniwa): {grouped}")
print(f"Średnia per kategoria(obliczona): {grouped.compute()}")

Średnia per kategoria(leniwa): Dask Series Structure:
npartitions=1
    float64
        ...
Dask Name: getitem, 5 expressions
Expr=((ArrowStringConversion(frame=FromMapProjectable(1626709))[['value', 'category']]).mean(observed=False, chunk_kwargs={'numeric_only': False}, aggregate_kwargs={'numeric_only': False}, _slice='value'))['value']
Średnia per kategoria(obliczona): category
A    100.006379
B    100.014957
C     99.999990
Name: value, dtype: float64


In [8]:
print(f"Liczba partycji: {ddf.npartitions}")
print(f"rozmiar danych (szacowany): {ddf.memory_usage(deep=True).compute()}")

Liczba partycji: 2
rozmiar danych (szacowany): Index            264
id          40000000
value       40000000
category    45000000
dtype: int64
